In [1]:
from pyhive import hive
import pymysql
conn = hive.Connection(host='127.0.0.1', port=10000, username='hadoop', database='comex')
cursor = conn.cursor()

In [2]:
def exec_sql(sql):
    cursor.execute(sql)
    for result in cursor.fetchall():
        print(result)

# 对缺失项进行预处理

# 实现hive和hbase映射

In [15]:
cursor.execute("DROP TABLE IF EXISTS test_01")

In [17]:
mapping_from_hbase_to_hive_sql = '''CREATE EXTERNAL TABLE test_01(id FLOAT,
name STRING,
man FLOAT,
woman FLOAT,
below20 FLOAT,
t_to_t4 FLOAT,
t5_to_t9 FLOAT,
th_to_th4 FLOAT,
th5_to_th9 FLOAT,
above40 FLOAT,
bachelor_degree_or_above FLOAT,
undergraduate_and_below FLOAT,
second_tier_city FLOAT,
fourth_tier_city FLOAT,
third_tier_citiy FLOAT,
first_tier_city FLOAT,
occupation1 STRING,
per1 FLOAT,
occupation2 STRING,
per2 FLOAT,
occupation3 STRING,
per3 FLOAT,
preference_film1 STRING,
pers1 FLOAT,
preference_film2 STRING,
pers2 FLOAT,
preference_film3 STRING,
pers3 FLOAT,
preference_film4 STRING,
pers4 FLOAT,
preference_film5 STRING,
pers5 FLOAT,
bought1 STRING,
bought2 STRING,
bought3 STRING,
bought4 STRING,
bought5 STRING,
bought6 STRING,
bought7 STRING,
bought8 STRING,
bought9 STRING,
bought10 STRING)STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler' 
WITH SERDEPROPERTIES ("hbase.columns.mapping" = ":key,info:id,
info:name,
info:man,
info:woman,
info:below20,
info:t_to_t4,
info:t5_to_t9,
info:th_to_th4,
info:th5_to_th9,
info:bachelor_degree_or_above,
info:undergraduate_and_below,
info:second_tier_city,
info:fourth_tier_city,
info:third_tier_citiy,
info:first_tier_city,
info:occupation1,
info:per1,
info:occupation2,
info:per2,
info:occupation3,
info:per3,
info:preference_film1,
info:pers1,
info:preference_film2,
info:pers2,
info:preference_film3,
info:pers3,
info:preference_film4,
info:pers4,
info:preference_film5,
info:pers5,
info:bought1,
info:bought2,
info:bought3,
info:bought4,
info:bought5,
info:bought6,
info:bought7,
info:bought8,
info:bought9,
info:bought10") 
TBLPROPERTIES("hbase.table.name" = "hbase_code_test")'''

In [19]:
#cursor.execute(mapping_from_hbase_to_hive_sql)

# ceate Hive table（将csv文件导入，后续改成hive和hbase映射）

In [3]:
cursor.execute("DROP TABLE IF EXISTS test")

In [4]:
#新建一个表，用于导入csv数据
create_Hive_table_sql = '''create table test(id FLOAT,
name STRING,
man FLOAT,
woman FLOAT,
below20 FLOAT,
t_to_t4 FLOAT,
t5_to_t9 FLOAT,
th_to_th4 FLOAT,
th5_to_th9 FLOAT,
above40 FLOAT,
bachelor_degree_or_above FLOAT,
undergraduate_and_below FLOAT,
second_tier_city FLOAT,
fourth_tier_city FLOAT,
third_tier_citiy FLOAT,
first_tier_city FLOAT,
occupation1 STRING,
per1 FLOAT,
occupation2 STRING,
per2 FLOAT,
occupation3 STRING,
per3 FLOAT,
preference_film1 STRING,
pers1 FLOAT,
preference_film2 STRING,
pers2 FLOAT,
preference_film3 STRING,
pers3 FLOAT,
preference_film4 STRING,
pers4 FLOAT,
preference_film5 STRING,
pers5 FLOAT,
bought1 STRING,
bought2 STRING,
bought3 STRING,
bought4 STRING,
bought5 STRING,
bought6 STRING,
bought7 STRING,
bought8 STRING,
bought9 STRING,
bought10 STRING) ROW format delimited fields terminated by ',' STORED AS TEXTFILE'''

In [5]:
cursor.execute(create_Hive_table_sql)

In [6]:
#将指定位置数据导入
input_sql = '''load data local inpath '/home/hadoop/Comex/personas.csv' into table test'''

In [7]:
cursor.execute(input_sql)

# after_prep（进行第一遍筛选，将带NULL值行全部删除）

In [8]:
cursor.execute("DROP TABLE IF EXISTS test_a")

In [9]:
#建表用于存放筛选后的数据
create_first_prep_sql= '''create table test_a(id FLOAT,
name STRING,
man FLOAT,
woman FLOAT,
below20 FLOAT,
t_to_t4 FLOAT,
t5_to_t9 FLOAT,
th_to_th4 FLOAT,
th5_to_th9 FLOAT,
above40 FLOAT,
bachelor_degree_or_above FLOAT,
undergraduate_and_below FLOAT,
second_tier_city FLOAT,
fourth_tier_city FLOAT,
third_tier_citiy FLOAT,
first_tier_city FLOAT,
occupation1 STRING,
per1 FLOAT,
occupation2 STRING,
per2 FLOAT,
occupation3 STRING,
per3 FLOAT,
preference_film1 STRING,
pers1 FLOAT,
preference_film2 STRING,
pers2 FLOAT,
preference_film3 STRING,
pers3 FLOAT,
preference_film4 STRING,
pers4 FLOAT,
preference_film5 STRING,
pers5 FLOAT,
bought1 STRING,
bought2 STRING,
bought3 STRING,
bought4 STRING,
bought5 STRING,
bought6 STRING,
bought7 STRING,
bought8 STRING,
bought9 STRING,
bought10 STRING) ROW format delimited fields terminated by ',' STORED AS TEXTFILE'''

In [10]:
cursor.execute(create_first_prep_sql)

In [11]:
#将筛选后的数据进行插入操作
insert_table_sql = '''insert overwrite table test_a select * from personas where id IS NOT NULL 
AND name IS NOT NULL
AND man IS NOT NULL 
AND woman IS NOT NULL 
AND below20 IS NOT NULL 
AND t_to_t4 IS NOT NULL 
AND t5_to_t9 IS NOT NULL 
AND th_to_th4 IS NOT NULL 
AND th5_to_th9 IS NOT NULL 
AND above40 IS NOT NULL 
AND bachelor_degree_or_above IS NOT NULL 
AND undergraduate_and_below IS NOT NULL 
AND second_tier_city IS NOT NULL 
AND fourth_tier_city IS NOT NULL 
AND third_tier_citiy IS NOT NULL 
AND first_tier_city IS NOT NULL 
AND occupation1 IS NOT NULL 
AND per1 IS NOT NULL 
AND occupation2 IS NOT NULL 
AND per2 IS NOT NULL 
AND occupation3 IS NOT NULL 
AND per3 IS NOT NULL 
AND preference_film1 IS NOT NULL 
AND pers1 IS NOT NULL 
AND preference_film2 IS NOT NULL 
AND pers2 IS NOT NULL 
AND preference_film3 IS NOT NULL 
AND pers3 IS NOT NULL 
AND preference_film4 IS NOT NULL 
AND pers4 IS NOT NULL 
AND preference_film5 IS NOT NULL 
AND pers5 IS NOT NULL 
AND bought1 IS NOT NULL 
AND bought2 IS NOT NULL 
AND bought3 IS NOT NULL 
AND bought4 IS NOT NULL 
AND bought5 IS NOT NULL 
AND bought6 IS NOT NULL 
AND bought7 IS NOT NULL 
AND bought8 IS NOT NULL 
AND bought9 IS NOT NULL 
AND bought10 IS NOT NULL'''

In [12]:
cursor.execute(insert_table_sql)

In [13]:
count_sql = 'select count(*) from test_a'
exec_sql(count_sql)

(1003,)


# 依据需求文档进行数据提取

# table_1_PUPsex

# table_2_CUPsex

# table_3_PUPage

# table_4_CUPage

# table_5_PUPedu

# table_6_PUPdistrict

# table_7_PMUjob

# table_8_CMUjob

# table_9_DataPiao

# table_10_DataClass

# table_11_PiaoClass

# 写进数据库

# 用pandas查看一下

In [7]:
#打开数据库连接
db= pymysql.connect(host="127.0.0.1",user="root",
     password="123456",db="comex",port=3306)

In [8]:
import pandas as pd

In [9]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [10]:
sql = 'select * from personas'
pd = pd.read_sql(sql,db)

In [11]:
pd.head(1)

,id,name,man,woman,below20,t_to_t4,t5_to_t9,th_to_th4,th5_to_th9,above40,bachelor_degree_or_above,undergraduate_and_below,second_tier_city,fourth_tier_city,third_tier_citiy,first_tier_city,occupation1,per1,occupation2,per2,occupation3,per3,preference_film1,pers1,preference_film2,pers2,preference_film3,pers3,preference_film4,pers4,preference_film5,pers5,bought1,bought2,bought3,bought4,bought5,bought6,bought7,bought8,bought9,bought10
0,345013.0,战神纪,0.51,0.49,0.04,0.26,0.28,0.18,0.12,0.11,0.31,0.69,0.41,0.24,0.2,0.16,白领,0.45,其他,0.38,学生,0.17,动作,0.51,爱情,0.14,喜剧,0.12,科幻,0.09,动画,0.06,红海行动,唐人街探案2,战狼2,复仇者联盟3：无限战争,羞羞的铁拳,捉妖记2,狂暴巨兽,前任3：再见前任,头号玩家,后来的我们
